In [47]:
# Importação das bibliotecas necessárias
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
from IPython.display import HTML

In [48]:
# Variáveis
prototxt = 'deploy.prototxt.txt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
confidence_custom = 0.5

In [49]:
# Carrega o modelo treinado
net = cv2.dnn.readNetFromCaffe(prototxt, model)

In [50]:
# Inicializa o stream do video
vs = VideoStream(src=0).start()
time.sleep(1)

In [51]:
# Percorre os frames do video
while True:
    
    # Redimensiona o tamanho do video
    frame = vs.read()
    frame = imutils.resize(frame, width=800)
 
    # Converte as dimensões do video em formato blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
        (300, 300), (104.0, 177.0, 123.0))
 
    # Envia o blob da image para a rede para ter acesso as predições e detecções
    net.setInput(blob)
    detections = net.forward()
        
    # Controla o número de faces detectadas
    num_faces = 0
    
    # Itera o número de faces encontradas
    for i in range(0, detections.shape[2]):
        
        # Nível de confiança para a detecção atual
        confidence = detections[0, 0, i, 2]

        # Filtra somente as detecções que tem uma confiança maior que a definida anteriomente
        if confidence < confidence_custom:
            continue
        
        # Incrementa o número de faces detectadas
        num_faces += 1

        # calcula as coordenadas X e Y para desenhar o "box" sobre a face detectada
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
 
        # Desenha o "box" sobre a face e também exibe o percentual de confiança sobre a detecção
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY),
            (0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
    # Mostra o número de faces detectadas no video
    cv2.putText(frame, "Quantidade Faces: {}".format(num_faces), 
        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Exibe o frame do video com as detecções
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # Controle de parada do video ao apertar tecla "q"
    if key == ord("q"):
        break

# Fecha as janelas e termina os processos de detecção
cv2.destroyAllWindows()
vs.stop()
vs.stream.release()

In [52]:
# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/MdbVAd0ABM8" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

Caso o link acima não exiba o video, acesse por aqui:
https://nbviewer.jupyter.org/github/willsilvano/computer-vision/blob/master/notebooks/projeto-01-deteccao-face/Detect%20faces%20on%20video.ipynb